In [3]:
# ✨ Cell 1 – Ping-resonator demo + wav preview ✨
# ------------------------------------------------------------------
# 0) Install once (does nothing if already present)
%pip install --quiet --upgrade numpy soundfile IPython

# 1) Imports
import numpy as np, soundfile as sf
from math import pi, sin, cos
from IPython.display import Audio, display

# 2) Little helpers -------------------------------------------------
def normalize(buf):           # avoid clipping
    return buf / (np.max(np.abs(buf)) + 1e-12)

def excite(duration_ms=5, fs=48_000, colour="white"):
    n = int(fs * duration_ms * 1e-3)
    return np.random.randn(n)  # white burst (good enough for a ping)

# 3) Modal-resonator object ----------------------------------------
class ModalResonator:
    def __init__(self, f0=200, t60=1.0, fs=48_000, Q=12):
        self.fs = fs
        self.z1 = self.z2 = 0.0
        self.set_mode(f0, t60, Q)

    # coefficient factory
    @staticmethod
    def _biquad_bandpass(f0, Q, fs, r):
        w0 = 2 * pi * f0 / fs
        alpha = sin(w0) / (2 * Q)
        b0, b1, b2 =  Q * alpha, 0.0, -Q * alpha
        a0, a1, a2 =  1 + alpha, -2 * cos(w0), 1 - alpha
        return [b0/a0, b1/a0, b2/a0,
                a1/a0 * r, a2/a0 * r]

    def set_mode(self, f0, t60, Q=12):
        r = 10 ** (-3 / (t60 * self.fs))          # −60 dB in t60 s
        self.b0, self.b1, self.b2, self.a1, self.a2 = \
            self._biquad_bandpass(f0, Q, self.fs, r)

    def process(self, x):
        y     = self.b0*x + self.z1
        self.z1 = self.b1*x - self.a1*y + self.z2
        self.z2 = self.b2*x - self.a2*y
        return y

# 4) Synthesize a ping ---------------------------------------------
fs   = 48_000
hit  = excite(5, fs)                  # 5 ms burst
reso = ModalResonator(f0=155, t60=1.4, fs=fs, Q=12)

Ntail = fs * 3                        # 3-second ring-out
out   = np.zeros(len(hit) + Ntail)
for n, x in enumerate(hit):
    out[n] = reso.process(x)
for n in range(len(hit), len(out)):
    out[n] = reso.process(0.0)

out = normalize(out)

# 5) Write wav + inline preview ------------------------------------
sf.write("ping.wav", out, fs)
print("Wrote ping.wav  ({} samples @ {} Hz)".format(len(out), fs))

display(Audio(out, rate=fs))   # widget you can click to hear

Note: you may need to restart the kernel to use updated packages.
Wrote ping.wav  (144240 samples @ 48000 Hz)
